In [39]:
import tensorflow as tf
import os
import cv2
import numpy as np
from tensorflow.keras.applications.inception_v3 import preprocess_input
import matplotlib.pyplot as plt


In [40]:
model=tf.keras.models.load_model('classifier.h5')

In [52]:
def mask_detector(configure_file, model_file, model, img_path, mask):
    '''
    In this function we are Loading our prototxt and cafe model. 
    Then we load the image, extract the dimensions, resize to 300px and create a blob using DNN.
    Made prediction using our classifier and displayed output as text in image.
    '''
    
    #laod model
    net=cv2.dnn.readNetFromCaffe(configure_file,model_file)
    
    #read image
    image=cv2.imread(img_path)
    
    #extracting dimensions
    h,w=image.shape[:2]
    
    #preprocessing image
    
    blob=cv2.dnn.blobFromImage(image,scalefactor = 1.0, size = (300,300), mean = (104.0, 177.0, 123.0))
    
    #passing blob through the network to detect and prediction
    net.setInput(blob)
    detections=net.forward()
    
    #we’ll loop over the detections and draw boxes around the detected faces with confidence >0.5
    for i in range(detections.shape[2]):
     
        confidence=detections[0,0,i,2]
        if confidence>0.25:
            #detection height and width w.r.t image is alwayas within 0 and 1
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            
            #making sure starting and ending pixels lie inside image
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            
            #extracting face
            face = image[startY:endY, startX:endX]
            
            #preprocessing face for classifer model
            face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face=cv2.resize(face,(150,150))
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
            pred = model.predict(face) #predicting mask or no mask by passing extracted face from image

            proba=1-pred #as we have class 0 for maks and 1 for no mask
            print(proba)

            label = "Mask" if pred < 0.4 else "No Mask"

            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
            label=label+' '+str((proba[0][0]*100).round(3))+'%'

            cv2.putText(image, label, (startX, startY-10),cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 1)
            cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
            mask.append(image)
    image=cv2.resize(image,(700,500))
    saving_path = 'mask5.jpg'
    cv2.imwrite(saving_path, image)
    cv2.imshow("prediction", image)

    cv2.waitKey(0)
            

    

In [53]:
configure_file=r'face_detector\deploy.prototxt' #this file contain model architecture
model_file=r'face_detector\res10_300x300_ssd_iter_140000.caffemodel' #this file conatin model weight
img_path=r'C:\Users\Saket\Desktop\all\openCV\mask6.jpg'
mask=[]

In [54]:
mask_detector(configure_file, model_file, model, img_path, mask)

[[0.01142728]]
[[0.99999994]]
